In [1]:
from matplotlib import pyplot
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import numba
from numba import jit

In [2]:
@jit
def Fc1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=-((u[1:-1,1:-1]+u[1:-1,2:])**2-(u[1:-1,1:-1]+u[1:-1,:-2])**2)/(4*dx)-\
                  ((u[1:-1,1:-1]+u[2:,1:-1])*(v[1:-1,2:]+v[1:-1,1:-1])-\
                  (u[1:-1,1:-1]+u[:-2,1:-1])*(v[:-2,2:]+v[:-2,1:-1]))/(4*dy)
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [3]:
@jit
def Fv1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=((u[1:-1,2:]-2*u[1:-1,1:-1]+u[1:-1,:-2])/dx**2+\
                 (u[2:,1:-1]-2*u[1:-1,1:-1]+u[:-2,1:-1])/dy**2)/100
    
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [4]:
@jit
def Fp1(p,dx,dy):
    F=numpy.zeros(numpy.shape(p))
    F[1:-1,1:-1]=-(p[1:-1,2:]-p[1:-1,1:-1])/dx
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [5]:
@jit
def Fc2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=-((u[2:,1:-1]+u[1:-1,1:-1])*(v[1:-1,1:-1]+v[1:-1,2:])-\
                   (u[2:,:-2]+u[1:-1,:-2])*(v[1:-1,1:-1]+v[1:-1,:-2]))/(4*dx)-\
                  ((v[1:-1,1:-1]+[v[2:,1:-1]])**2-(v[1:-1,1:-1]+v[:-2,1:-1])**2)/(4*dy)
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [6]:
@jit
def Fv2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=((v[1:-1,2:]-2*v[1:-1,1:-1]+v[1:-1,:-2])/dx**2+\
                 (v[2:,1:-1]-2*v[1:-1,1:-1]+v[:-2,1:-1])/dy**2)/100
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [7]:
@jit
def Fp2(p,dx,dy):
    F=numpy.zeros((numpy.shape(p)))
    F[1:-1,1:-1]=-(p[2:,1:-1]-p[1:-1,1:-1])/dy
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [8]:
@jit
def poisson(f,omega):
    nj,ni=numpy.shape(f)-numpy.array((1,1))
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(-dy/2,1+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    p=numpy.zeros(numpy.shape(f))
    res=numpy.zeros(numpy.shape(f))
    
    res_max=1e3
    
    while res_max>1e-6:
        
        for j in range(1,nj):
            for i in range(1,ni):
                p[j,i]=omega/(2*(dx**2+dy**2))*((p[j,i-1]+p[j,i+1])*dy**2+\
                       (p[j-1,i]+p[j+1,i])*dx**2-f[j,i]*dx**2*dy**2)+\
                       (1-omega)*p[j,i]
        p[0,:]=-p[1,:]
        p[-1,:]=-p[-2,:]
        p[:,0]=-p[:,1]
        p[:,-1]=-p[:,-2]
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(p[j,i-1]-2*p[j,i]+p[j,i+1])/dx**2-\
                                  (p[j-1,i]-2*p[j,i]+p[j+1,i])/dy**2)
        res_max=numpy.max(res)
    
    return p

In [9]:
@jit
def getuv(w1,w2,p,dx,dy,dt):
    u=w1+Fp1(p,dx,dy)*dt
    v=w2+Fp2(p,dx,dy)*dt
    
    u[0,:]=-u[1,:]
    u[-1,:]=2-u[-2,:]
    u[:,0]=0
    u[:,-2]=0
    u[:,-1]=-u[:,-3]

    v[:,0]=-v[:,1]
    v[:,-1]=-v[:,-2]
    v[0,:]=0
    v[-2,:]=0
    v[-1,:]=-v[-3,:]
    
    return u,v

In [10]:
@jit
def difference(p,pold):
    nj,ni=numpy.shape(p)-numpy.array((1,1))
    diff=numpy.zeros(numpy.shape(p))
    for j in range(nj+1):
        for i in range(ni+1):
            diff[j,i]=numpy.abs(p[j,i]-pold[j,i])
    diff_max=numpy.max(diff)
    return diff_max

In [11]:
def grid_p(ni,nj):
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(-dy/2,1+dy/2,nj+1)
    X,Y=numpy.meshgrid(x,y)
    return X,Y

def grid_u(ni,nj):
    dx=1./(ni-1)
    dy=1./(nj-1)
    xu=numpy.linspace(0,1+dx,ni+1)
    yu=numpy.linspace(-dy/2,1+dy/2,nj+1)
    X,Y=numpy.meshgrid(xu,yu)
    return X,Y

def grid_v(ni,nj):
    dx=1./(ni-1)
    dy=1./(nj-1)
    xv=numpy.linspace(-dx/2,1+dx/2,ni+1)
    yv=numpy.linspace(0,1+dy,nj+1)
    X,Y=numpy.meshgrid(xv,yv)
    return X,Y

In [12]:
@jit
def RK3(ni,nj,dt):
    
    dx=1./(ni-1)
    dy=1./(nj-1)
    x=numpy.linspace(-dx/2,1+dx/2,ni+1)
    y=numpy.linspace(-dy/2,1+dy/2,nj+1)
    xu=numpy.linspace(0,1+dx,ni+1)
    yu=numpy.linspace(-dy/2,1+dy/2,nj+1)
    xv=numpy.linspace(-dx/2,1+dx/2,ni+1)
    yv=numpy.linspace(0,1+dy,nj+1)

    X,Y=numpy.meshgrid(x,y)
    Xu,Yu=numpy.meshgrid(xu,yu)
    Xv,Yv=numpy.meshgrid(xv,yv)

    u0=numpy.zeros((nj+1,ni+1))
    v0=numpy.zeros((nj+1,ni+1))
    u0[-1,1:-3]=2.
    p0=numpy.zeros((nj+1,ni+1))
    omega0=numpy.zeros((nj+1,ni+1))
            
    u=u0.copy()
    v=v0.copy()
    p=p0.copy()
    omega=omega0.copy()
    
    G1=numpy.zeros((nj+1,ni+1))
    G2=numpy.zeros((nj+1,ni+1))
    w1=numpy.zeros((nj+1,ni+1))
    w2=numpy.zeros((nj+1,ni+1)) 
    f=numpy.zeros((nj+1,ni+1))
    T=4.
    t=0.
    nt=0
    uold=numpy.ones((nj+1))
    
    
    while t<T:
        uold=u.copy()
        vold=v.copy()
        pold=p.copy()
        
        #from t to t+dt/3
        #t+=dt/3
        G1=Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt/3*G1
        w2=v+dt/3*G2
        f[1:-1,1:-1]=3./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1.85)
        u,v=getuv(w1,w2,p,dx,dy,dt/3)
        
        #from t+dt/3 to t+3dt/4
        #t+=5/12*dt
        G1=-5./9*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-5./9*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*15./16*G1
        w2=v+dt*15./16*G2
        f[1:-1,1:-1]=12./5/dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)        
        p=poisson(f,1.85)
        u,v=getuv(w1,w2,p,dx,dy,5*dt/12)
        
        #from t+3dt/4 to t+dt
        #t+=dt/4
        G1=-153./128*G1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
        G2=-153./128*G2+Fc2(u,v,dx,dy)+Fv2(u,v,dx,dy)
        w1=u+dt*8./15*G1
        w2=v+dt*8./15*G2
        f[1:-1,1:-1]=4./dt*((w1[1:-1,1:-1]-w1[1:-1,:-2])/dx+(w2[1:-1,1:-1]-w2[:-2,1:-1])/dy)
        p=poisson(f,1.85)
        u,v=getuv(w1,w2,p,dx,dy,dt/4)
        
        if nt%100==0:
            print(nt,difference(u,uold))
        nt+=1
        t+=dt
        
    psi=numpy.zeros((nj+1,ni+1))
    for j in range(1,nj+1):
        for i in range(ni+1):
            psi[j,i]=psi[j-1,i]+u[j,i]*dy
        
    return u,v,p,psi

In [ ]:
u,v,p,psi=RK3(129,129,1e-3)

In [ ]:
X,Y=grid_p(129,129)
Xu,Yu=grid_u(129,129)
Xv,Yv=grid_v(129,129)

pyplot.figure(figsize=(6.5,5))
pyplot.contour(X[1:-2,1:-2],Y[1:-2,1:-2],psi1[1:-2,1:-2],12)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.colorbar()
pyplot.quiver(X[1:-2:8,1:-2:8],Y[1:-2:8,1:-2:8],u1[1:-2:8,1:-2:8],v1[1:-2:8,1:-2:8],scale=5)

In [ ]:
psimin=1.
#find the minimum psi
for j in range(65,90):
    for i in range(65,90):
        if psimin>psi[j,i]:
            psimin=psi[j,i]
            imin=i
            jmin=j


In [ ]:
uc=u[:,imin]
vc=v[jmin,:]
#find average value of each two points 
uver=numpy.zeros(129)
for j in range(129):
    uver[j]=(uc[j+1]+uc[j])/2

vhor=numpy.zeros(129)
for i in range(129):
    vhor[i]=(vc[i+1]+vc[i])/2

In [ ]:
#point no. and u,v values in the reference 
yuc=numpy.array([0,7,8,9,13,22,36,58,64,79,94,109,122,123,124,125,128])
xvc=numpy.array([0,8,9,10,12,20,29,30,64,103,110,116,121,122,123,124,128])
uref=numpy.array([0.,-0.03717,-0.04192,-0.04775,-0.06434,-0.10150,-0.15662,\
                 -0.21090,-0.20581,-0.13641,0.00332,0.23151,0.68717,0.73722,\
                0.78871,0.84123,1.])
vref=numpy.array([0.,0.09233,0.10091,0.10890,0.12317,0.16077,0.17507,0.17527,\
                 0.05454,-0.24533,-0.22445,-0.16914,-0.10313,-0.08864,-0.07391,\
                 -0.05906,0.])

In [ ]:
#corresponding point u,v values during my computation
ucver=numpy.zeros(numpy.shape(uref))
for j in range(17):
    ucver[j]=uver[yuc[j]]

vchor=numpy.zeros(numpy.shape(vref))
for i in range(17):
    vchor[i]=vhor[xvc[i]]

In [ ]:
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('grid pt. no.')
pyplot.ylabel('$u$')
pyplot.plot(yuc,ucver,color='b',marker='o')
pyplot.plot(yuc,uref,color='r',marker='x')
pyplot.title('Results for u-velocity along Vertical Line through Geometric Center of Cavity',fontsize=18)
pyplot.legend(['Computation result','Reference result'],loc='best');

In [ ]:
pyplot.figure(figsize=(10,6))
pyplot.grid(True)
pyplot.xlabel('grid pt. no.')
pyplot.ylabel('$v$')
pyplot.plot(xvc,vchor,color='b',marker='o')
pyplot.plot(xvc,vref,color='r',marker='x')
pyplot.title('Results for u-Velocity along Horizontal Line through Geometric Center of Cavity ',fontsize=18)
pyplot.legend(['Computation result','Reference result'],loc='best');